In [1]:
import tensorflow as tf
import numpy as np

In [252]:
DATA_DIR='./data/shakespeare.txt'
BATCH_SIZE=100
LAYER_NUM=2
SEQ_LENGTH=100
HIDDEN_DIM=128
GENERATE_LEN=500
NB_EPOCH=150
MODE='train'
WEIGHTS=''
LEARNING_RATE=2e-3

In [253]:
def get_text_data():
    with open(DATA_DIR,"r") as text_file:
        data=text_file.read()
        data=data.replace('\n','')
        data=data.replace('$','')
    lis=list(set(data))    
    total_char,unique_char=len(data),len(lis)
    print ('data has %d characters, %d unique.' %(total_char, unique_char))
    char_to_ix = { ch:i for i,ch in enumerate(lis) }
    ix_to_char = { i:ch for i,ch in enumerate(lis) }
    return total_char,unique_char,char_to_ix,ix_to_char,data

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [254]:
total_chars,unique_chars,char_id,id_char,data=get_text_data()

data has 1075393 characters, 63 unique.


In [255]:
print(char_id)

{'I': 0, 'a': 1, 'C': 2, 'z': 3, 'r': 4, ';': 5, 'K': 6, 'V': 7, 'H': 8, 'n': 9, 'L': 10, 'd': 11, 'Y': 12, 'Q': 13, ':': 14, 'k': 15, 'g': 16, 'F': 17, 'G': 18, 'E': 19, 'w': 20, 'O': 21, 't': 22, '-': 23, '&': 24, 'o': 25, 'X': 26, 'y': 27, '!': 28, 'l': 29, 'p': 30, 'h': 31, 'q': 32, '3': 33, 'T': 34, '.': 35, 'R': 36, ',': 37, 'S': 38, 'N': 39, 'c': 40, 'j': 41, 'B': 42, 'W': 43, 'f': 44, 's': 45, ' ': 46, 'b': 47, 'i': 48, 'm': 49, 'e': 50, 'M': 51, 'P': 52, 'A': 53, "'": 54, '?': 55, 'D': 56, 'Z': 57, 'u': 58, 'v': 59, 'J': 60, 'x': 61, 'U': 62}


In [256]:
def generate_next_batch(seq_length,p):
    inputs = [char_id[ch] for ch in data[p:p+seq_length]]
    targets = [char_id[ch] for ch in data[p+1:p+seq_length+1]]
    inputs=np.array(inputs).reshape(seq_length,1)
    targets=np.array(targets).reshape(seq_length,1)
    return inputs,targets

In [257]:
#test generator function here
def generator(seed_i,n):
    x=np.zeros((1,1))
    x[0][0]= seed_i
    ixes=[]
    ixes.append(seed_i[0])
    for t in range(n):
        p=sess.run(prediction,{X:x})
        ix = np.random.choice(range(unique_chars), p=p.ravel())
        x[0][0]=ix
        ixes.append(ix)
    return ixes

In [258]:
def build_rnn(x):
    keep_prob = tf.placeholder_with_default(1.0, shape=())
    with tf.name_scope('rnn'):
        cells= [tf.contrib.rnn.BasicRNNCell(num_units=HIDDEN_DIM,activation=tf.nn.tanh)
            for layer in range(LAYER_NUM)]
        cells_drop = [tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
              for cell in cells]
        multi_layer_cell = tf.contrib.rnn.MultiRNNCell(cells_drop)
        outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, x, dtype=tf.float32)
        return outputs,states

In [259]:
def build_LSTM(x):
    keep_prob = tf.placeholder_with_default(0.2, shape=())
    with tf.name_scope('lstm'):
        cells= [tf.contrib.rnn.BasicLSTMCell(num_units=HIDDEN_DIM,activation=tf.nn.relu)
            for layer in range(LAYER_NUM)]
        cells_drop = [tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=keep_prob)
              for cell in cells]
        multi_layer_cell = tf.contrib.rnn.MultiRNNCell(cells)
        outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, x, dtype=tf.float32)
        return outputs,states

In [260]:
reset_graph()

X=tf.placeholder(tf.uint8,[None,1])
Y=tf.placeholder(tf.uint8,[None,1])

X_onehot=tf.one_hot(X,unique_chars)
Y_onehot=tf.one_hot(Y,unique_chars)

outputs,states=build_LSTM(X_onehot)
outputs=tf.transpose(outputs,perm=[1,0,2])

W=tf.Variable(tf.truncated_normal((HIDDEN_DIM,unique_chars),-1,1))
B=tf.Variable(tf.ones((1,unique_chars)))
Ys=tf.matmul(outputs[0],W)+B

prediction=tf.nn.softmax(Ys)
xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y_onehot, logits=Ys)
loss = tf.reduce_mean(xentropy)
with tf.name_scope('adam_optimizer'):
    optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
    minimize = optimizer.minimize(loss)

In [261]:
saver=tf.train.Saver()
init=tf.global_variables_initializer()

In [262]:
no_of_batches=int(total_chars//BATCH_SIZE)
ptr=0
# with tf.Session() as sess:
#     smooth_loss = -np.log(1.0/unique_chars)*BATCH_SIZE # loss at iteration 0
#     sess.run(init)
#     for i in range(NB_EPOCH):
#         for j in range(no_of_batches):
#             if ptr+BATCH_SIZE+1 >= len(data): 
#                 ptr=0
#             inputs,targets=generate_next_batch(BATCH_SIZE,ptr)
#             ptr+=BATCH_SIZE
#             if(j%100==0):
#                 loss_t=sess.run(loss,feed_dict={X:inputs, Y:targets}) 
#                 sample_ix=generator(inputs[0],200)
#                 txt = ''.join(id_char[ix] for ix in sample_ix)
#                 smooth_loss = smooth_loss * 0.999 + loss_t * 0.001
#                 print('epoch %i iteration %i loss %f'%(i,j,smooth_loss))
#                 print ('----\n %s \n----' % (txt, ))
#             sess.run(minimize,feed_dict={X:inputs,Y:targets})
#     save_path = saver.save(sess, "/tmp/RNNmodel.ckpt")

In [263]:
# with tf.Session() as sess:
#     smooth_loss = -np.log(1.0/unique_chars)*BATCH_SIZE # loss at iteration 0
#     sess.run(init)
#     for i in range(NB_EPOCH):
#         for j in range(no_of_batches):
#             if ptr+BATCH_SIZE+1 >= len(data): 
#                 ptr=0
#             inputs,targets=generate_next_batch(BATCH_SIZE,ptr)
#             ptr+=BATCH_SIZE
#             if(j%100==0):
#                 loss_t=sess.run(loss,feed_dict={X:inputs, Y:targets}) 
#                 sample_ix=generator(inputs[0],200)
#                 txt = ''.join(id_char[ix] for ix in sample_ix)
#                 smooth_loss = smooth_loss * 0.999 + loss_t * 0.001
#                 print('epoch %i iteration %i loss %f'%(i,j,smooth_loss))
#                 print ('----\n %s \n----' % (txt, ))
#             sess.run(minimize,feed_dict={X:inputs,Y:targets})
#         if (i%5==0):
#             save_path = saver.save(sess, "../LSTMmodel.ckpt")

In [268]:
with tf.Session() as sess:
    smooth_loss = -np.log(1.0/unique_chars)*BATCH_SIZE # loss at iteration 0
    sess.run(init)
    for i in range(NB_EPOCH):
        for j in range(no_of_batches):
            if ptr+BATCH_SIZE+1 >= len(data): 
                ptr=0
            inputs,targets=generate_next_batch(SEQ_LENGTH,ptr)
            ptr+=BATCH_SIZE
            if(j%100==0):
                loss_t=sess.run(loss,feed_dict={X:inputs, Y:targets}) 
                sample_ix=generator(inputs[0],150)
                txt = ''.join(id_char[ix] for ix in sample_ix)
                smooth_loss = smooth_loss * 0.999 + loss_t * 0.001
                print('epoch %i iteration %i loss %f'%(i,j,smooth_loss))
                print ('----\n %s \n----' % (txt, ))
            sess.run(minimize,feed_dict={X:inputs,Y:targets})
        if (i%5==0):
            save_path = saver.save(sess, "../LSTMmodel2.ckpt")